In [1]:
# train_resnet.py
import os
import glob
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from torchvision import models


In [2]:
# --- 설정 ---
TRAIN_DIR = '/home/work/AIprogramming/Ai_proj/data_generator/train_data'
TEST_DIR  = '/home/work/AIprogramming/Ai_proj/data_generator/test_data'
CHECKPOINT_DIR = './checkpoints'
HISTORY_CSV    = './history.csv'
BATCH_SIZE = 128
EPOCHS     = 30
LR         = 1e-3
DEVICE     = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

os.makedirs(CHECKPOINT_DIR, exist_ok=True)
def load_pt_folder(pt_dir):
    """폴더의 .pt 파일 전부 읽어서 하나의 TensorDataset으로 반환"""
    imgs_list, labels_list = [], []
    for path in sorted(glob.glob(os.path.join(pt_dir, '*.pt'))):
        print(f'Loading {path} ...')
        imgs, lbls = torch.load(path)
        imgs_list.append(imgs)
        labels_list.append(lbls)
    imgs = torch.cat(imgs_list, dim=0)
    lbls = torch.cat(labels_list, dim=0)
    return TensorDataset(imgs, lbls)



In [3]:
# 1) 데이터 준비
train_ds = load_pt_folder(TRAIN_DIR)
test_ds  = load_pt_folder(TEST_DIR)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

# 2) 모델 준비 (ResNet-18, CIFAR10용으로 마지막 레이어 수정)
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(DEVICE)

# 3) 손실/최적화 준비
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3, factor=0.5)

# 4) 학습 루프
history = []
best_acc = 0.0


Loading /home/work/AIprogramming/Ai_proj/data_generator/train_data/clean.pt ...
Loading /home/work/AIprogramming/Ai_proj/data_generator/train_data/fgsm.pt ...
Loading /home/work/AIprogramming/Ai_proj/data_generator/train_data/mifgsm.pt ...
Loading /home/work/AIprogramming/Ai_proj/data_generator/train_data/pgd.pt ...
Loading /home/work/AIprogramming/Ai_proj/data_generator/train_data/rfgsm.pt ...
Loading /home/work/AIprogramming/Ai_proj/data_generator/test_data/clean.pt ...
Loading /home/work/AIprogramming/Ai_proj/data_generator/test_data/cw.pt ...
Loading /home/work/AIprogramming/Ai_proj/data_generator/test_data/fgsm.pt ...
Loading /home/work/AIprogramming/Ai_proj/data_generator/test_data/mifgsm.pt ...
Loading /home/work/AIprogramming/Ai_proj/data_generator/test_data/pgd.pt ...
Loading /home/work/AIprogramming/Ai_proj/data_generator/test_data/rfgsm.pt ...


/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:207: UserWarning: The parameter 'pretrained' is deprecated, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:220: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:
for epoch in range(1, EPOCHS+1):
    # --- train ---
    model.train()
    running_loss = 0.0
    running_correct = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)
        running_correct += (outputs.argmax(1) == labels).sum().item()
    train_loss = running_loss / len(train_loader.dataset)
    train_acc  = running_correct / len(train_loader.dataset)

    # --- validate ---
    model.eval()
    val_loss = 0.0
    val_correct = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * imgs.size(0)
            val_correct += (outputs.argmax(1) == labels).sum().item()
    val_loss /= len(test_loader.dataset)
    val_acc  = val_correct / len(test_loader.dataset)

    # --- 스케줄러 스텝 & 체크포인트 ---
    scheduler.step(val_acc)
    is_best = val_acc > best_acc
    if is_best:
        best_acc = val_acc
        torch.save(model.state_dict(), os.path.join(CHECKPOINT_DIR, 'best_resnet18.pth'))

    # --- 히스토리 기록 ---
    print(f"Epoch {epoch:02d}/{EPOCHS} | "
          f"Train: loss={train_loss:.4f}, acc={train_acc:.4f} | "
          f"Val:   loss={val_loss:.4f}, acc={val_acc:.4f} {'*' if is_best else ''}")
    history.append({
        'epoch': epoch,
        'train_loss': train_loss,
        'train_acc': train_acc,
        'val_loss': val_loss,
        'val_acc': val_acc
    })


Epoch 01/30 | Train: loss=1.0335, acc=0.6343 | Val:   loss=0.9926, acc=0.6617 *


/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


Epoch 02/30 | Train: loss=0.5818, acc=0.7975 | Val:   loss=0.9916, acc=0.6913 *
Epoch 03/30 | Train: loss=0.3383, acc=0.8820 | Val:   loss=1.2626, acc=0.6806 
Epoch 04/30 | Train: loss=0.2042, acc=0.9287 | Val:   loss=1.2716, acc=0.6960 *
Epoch 05/30 | Train: loss=0.1401, acc=0.9511 | Val:   loss=1.5534, acc=0.6791 
Epoch 06/30 | Train: loss=0.1073, acc=0.9628 | Val:   loss=1.5710, acc=0.6863 
Epoch 07/30 | Train: loss=0.0879, acc=0.9697 | Val:   loss=1.6469, acc=0.6898 
Epoch 08/30 | Train: loss=0.0729, acc=0.9745 | Val:   loss=1.6257, acc=0.6993 *
Epoch 09/30 | Train: loss=0.0626, acc=0.9784 | Val:   loss=1.6835, acc=0.7017 *
Epoch 10/30 | Train: loss=0.0559, acc=0.9808 | Val:   loss=1.8116, acc=0.6987 
Epoch 11/30 | Train: loss=0.0535, acc=0.9817 | Val:   loss=1.6705, acc=0.7011 
Epoch 12/30 | Train: loss=0.0445, acc=0.9848 | Val:   loss=1.8421, acc=0.6986 
Epoch 13/30 | Train: loss=0.0445, acc=0.9848 | Val:   loss=1.8191, acc=0.7026 *
Epoch 14/30 | Train: loss=0.0397, acc=0.9863 | 

In [ ]:
# 5) 히스토리 저장
pd.DataFrame(history).to_csv(HISTORY_CSV, index=False)
print("Training complete. Best val_acc:", best_acc)
print("Checkpoint and history saved.")


In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# (1) 라이브러리 임포트
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# ─────────────────────────────────────────────────────────────────────────────
# (2) history.csv 읽기
history = pd.read_csv('history.csv')

# ─────────────────────────────────────────────────────────────────────────────
# (3) Figure & Subplots 생성
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 3-1) 손실(Loss) 곡선
axes[0].plot(history['epoch'], history['train_loss'], label='Train Loss', marker='o')
axes[0].plot(history['epoch'], history['val_loss'],   label='Val   Loss', marker='s')
axes[0].set_title('Loss vs. Epoch')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True)

# 3-2) 정확도(Accuracy) 곡선
axes[1].plot(history['epoch'], history['train_acc'], label='Train Acc', marker='o')
axes[1].plot(history['epoch'], history['val_acc'],   label='Val   Acc', marker='s')
axes[1].set_title('Accuracy vs. Epoch')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()
# ─────────────────────────────────────────────────────────────────────────────
